# Playground

In [1]:
from typing import List, Set, Dict, Tuple, Optional, Any
from collections import defaultdict

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import math 
import torch
from torch import nn, Tensor
from torch.nn.functional import softplus, relu
from torch.distributions import Distribution, Normal
from torch.utils.data import DataLoader

from gmfpp.utils.data_preparation import *
from gmfpp.utils.data_transformers import *
from gmfpp.utils.plotting import *

from gmfpp.models.ReparameterizedDiagonalGaussian import *
from gmfpp.models.CytoVariationalAutoencoder import *
from gmfpp.models.VariationalAutoencoder import *
from gmfpp.models.ConvVariationalAutoencoder import *
from gmfpp.models.VariationalInference import *

%matplotlib inline

In [2]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

## Load data

In [3]:
metadata = read_metadata("./data/tiny/metadata.csv")

In [4]:
relative_path = get_relative_image_paths(metadata)
image_paths = ["./data/tiny/" + path for path in relative_path]
images = load_images(image_paths)

In [5]:
len(images)

259

## VAE

In [6]:
train_set = prepare_raw_images(images)
normalize_channels_inplace(train_set)
print(train_set.shape)

torch.Size([259, 3, 68, 68])


In [7]:
channel_first = view_channel_dim_first(train_set)
for i in range(channel_first.shape[0]):
    channel = channel_first[i]
    print("channel {} interval: [{:.2f}; {:.2f}]".format(i, torch.min(channel), torch.max(channel)))

channel 0 interval: [0.02; 1.00]
channel 1 interval: [0.04; 1.00]
channel 2 interval: [0.05; 1.00]


In [8]:
# VAE
image_shape = np.array([3, 68, 68])
latent_features = 256
vae = CytoVariationalAutoencoder(image_shape, latent_features)
#vae = VariationalAutoencoder(image_shape, latent_features)

beta = 1
vi = VariationalInference(beta=beta)

# The Adam optimizer works really well with VAEs.
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-2, weight_decay=10e-4)

# define dictionary to store the training curves
training_data = defaultdict(list)
validation_data = defaultdict(list)

In [ ]:
num_epochs = 1000
batch_size = 16

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f">> Using device: {device}")

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)

# move the model to the device
vae = vae.to(device)

# training..

for epoch in range(num_epochs):
    print(f"epoch: {epoch}/{num_epochs}")    

    training_epoch_data = defaultdict(list)
    vae.train()

    for x in train_loader:
        x = x.to(device)
        
        # perform a forward pass through the model and compute the ELBO
        loss, diagnostics, outputs = vi(vae, x)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(vae.parameters(), 10_000)
        optimizer.step()

        # gather data for the current batch
        for k, v in diagnostics.items():
            training_epoch_data[k] += [v.mean().item()]

    print("training | elbo: {:2f}, log_px: {:.2f}, kl: {:.2f}:".format(np.mean(training_epoch_data["elbo"]), np.mean(training_epoch_data["log_px"]), np.mean(training_epoch_data["kl"])))

    # gather data for the full epoch
    for k, v in training_epoch_data.items():
        training_data[k] += [np.mean(training_epoch_data[k])]

    # Evaluate on a single batch, do not propagate gradients
    with torch.no_grad():
        vae.eval()

        # Just load a single batch from the test loader
        '''x, y = next(iter(test_loader))'''
        x = x.to(device)

        # perform a forward pass through the model and compute the ELBO
        loss, diagnostics, outputs = vi(vae, x)

        # gather data for the validation step
        for k, v in diagnostics.items():
            validation_data[k] += [v.mean().item()]

    print("validation | elbo: {:2f}, log_px: {:.2f}, kl: {:.2f}:".format(np.mean(validation_data["elbo"]), np.mean(validation_data["log_px"]), np.mean(validation_data["kl"])))    

>> Using device: cpu
epoch: 0/1000
training | elbo: -13455.945923, log_px: -12861.16, kl: 594.79:
validation | elbo: -12656540.000000, log_px: -12588970.00, kl: 67570.55:
epoch: 1/1000
training | elbo: -12837.380310, log_px: -12564.46, kl: 272.92:
validation | elbo: -6334754.938477, log_px: -6300692.52, kl: 34062.69:
epoch: 2/1000
training | elbo: -12549.311401, log_px: -12351.60, kl: 197.71:
validation | elbo: -4227277.443034, log_px: -4204527.34, kl: 22750.29:
epoch: 3/1000
training | elbo: -12121.643494, log_px: -11971.73, kl: 149.91:
validation | elbo: -3173421.459229, log_px: -3156323.25, kl: 17098.34:
epoch: 4/1000
training | elbo: -11437.510620, log_px: -11313.97, kl: 123.54:
validation | elbo: -2540923.657617, log_px: -2527223.03, kl: 13700.74:
epoch: 5/1000
training | elbo: -10397.099487, log_px: -10290.65, kl: 106.45:
validation | elbo: -2119069.248372, log_px: -2107640.90, kl: 11428.44:
epoch: 6/1000
training | elbo: -8885.671509, log_px: -8792.54, kl: 93.13:
validation | el

In [ ]:
plt.plot(training_data["elbo"])

## Compare reconstruction and original image

In [ ]:
x = train_set[0]

In [ ]:
plot_image(x)

In [ ]:
# vae.eval() # because of batch normalization
outputs = vae(x[None,:,:,:])
px = outputs["px"]

x_reconstruction = px.sample()
x_reconstruction = x_reconstruction[0]
plot_image_channels(x_reconstruction)

In [ ]:
plot_image_channels(x)

In [ ]:
x_reconstruction = px.sample()
x_reconstruction = x_reconstruction[0]
plot_image(clip_image_to_zero_one(x_reconstruction))